WCET
Interrupts
Get semantic into knuckle


Cryptol examples
Low* 


In [2]:
%%file /tmp/hello.c

int main(){
    return 3;
}

Writing /tmp/hello.c


In [4]:
!gcc /tmp/hello.c -o /tmp/hello

In [5]:
!objdump -d /tmp/hello | grep main.: -A20

0000000000001129 <main>:
    1129:	f3 0f 1e fa          	endbr64 
    112d:	55                   	push   %rbp
    112e:	48 89 e5             	mov    %rsp,%rbp
    1131:	b8 03 00 00 00       	mov    $0x3,%eax
    1136:	5d                   	pop    %rbp
    1137:	c3                   	ret    

Disassembly of section .fini:

0000000000001138 <_fini>:
    1138:	f3 0f 1e fa          	endbr64 
    113c:	48 83 ec 08          	sub    $0x8,%rsp
    1140:	48 83 c4 08          	add    $0x8,%rsp
    1144:	c3                   	ret    


In [1]:
import angr
import archinfo
arch = archinfo.ArchPcode("x86:LE:64:default")
proj = angr.Project("/tmp/hello",arch=arch, engine=angr.engines.UberEnginePcode)

WARNING  | 2025-01-03 11:10:44,200 | archinfo.arch_pcode | Unexpected SP conflict
WARNING  | 2025-01-03 11:10:44,336 | cle.backends.elf.elf | User specified <Arch x86:LE:64:default (LE)> but autodetected <Arch AMD64 (LE)>. Proceed with caution.
WARNING  | 2025-01-03 11:10:44,338 | cle.backends.elf.relocation | Unknown reloc 8 on x86:LE:64:default
WARNING  | 2025-01-03 11:10:44,339 | cle.backends.elf.relocation | Unknown reloc 6 on x86:LE:64:default


In [2]:
state = proj.factory.entry_state()
state.block().pp()

        _start:
401040  ENDBR64 
401044  XOR     EBP, EBP
401046  MOV     R9, RDX
401049  POP     RSI
40104a  MOV     RDX, RSP
40104d  AND     RSP, -0x10
401051  PUSH    RAX
401052  PUSH    RSP
401053  XOR     R8D, R8D
401056  XOR     ECX, ECX
401058  LEA     RDI, [main]
40105f  CALL    qwordptr[0x403fd8]


In [12]:
ins = proj.factory.block(0x401129, size=4)
ins

<Block for 0x401129, 4 bytes>

In [6]:
state = proj.factory.call_state(0x401129)
#state.step()
b = state.block()


In [46]:
state = proj.factory.call_state(0x401129)
s2 = state.step()
s2, = s2.successors
r = s2.registers
s2.regs.rax

WARNING  | 2025-01-03 09:48:55,915 | angr.storage.memory_mixins.default_filler_mixin | Filling register rbp with 8 unconstrained bytes referenced from 0x40112d (main+0x4 in hello (0x112d))


<BV64 0x3>

AttributeError: 'AngrObjectFactory' object has no attribute 'path'

Right. one of the breakpoints is address concretization. Their memory model is 

https://github.com/angr/angr/blob/master/angr/engines/pcode/emulate.py basic pcode interpreter
https://github.com/angr/angr/blob/master/angr/engines/pcode/behavior.py "behavior". There is a diciontary at the bottom

I can't believe the amount of indirection there is in here.

Ok, but even 

Maybe subclass the pcodenegine mixin.

In [83]:
b = angr.engines.pcode.behavior.BehaviorFactory()
from pypcode import OpCode
import claripy
claripy.backends.z3.convert(b.get_behavior_for_opcode(OpCode.BOOL_AND).evaluate_binary(1,1,claripy.BVV(1,8),claripy.BVV(1,8)))

1

In [81]:
x = claripy.BVS("x", 8) #.dbg_repr()
import claripy.backends.backend_z3
#claripy.backends.backend_z3.BackendZ3().convert(x)
claripy.backends.z3.convert(x)

x_165_8

In [47]:
state.inspect.

In [16]:
main_symbol = proj.loader.main_object.get_symbol("main")
main_symbol

<Symbol "main" in hello at 0x401129>

pywasm https://github.com/mohanson/pywasm
https://github.com/AntonLydike/riscemu

https://github.com/codypierce/pyemu x86
https://github.com/ForceBru/PyVM

https://github.com/mnaberez/py65 6502

In [14]:
import riscemu

from riscemu.riscemu_main import RiscemuMain, RunConfig

# Create an instance of the RiscemuMain class
emulator = RiscemuMain()

# Set the configuration (this is just an example; set according to your needs)
config = RunConfig(stack_size=8192, verbosity=1, flen=64)
emulator.cfg = config

# Set the input files (example assembly files)
emulator.input_files = ["path/to/your/assembly/file.asm"]

# Register all available instruction sets and program loaders
emulator.register_all_isas()
emulator.register_all_program_loaders()

# Call the run method to start the emulator
emulator.run()
cpu = UserModeCPU(self.selected_ins_sets, self.cfg)
cpu.setup_stack(self.cfg.stack_size)
cpu.launch(self.cfg.verbosity > 1)

AssertionError: self.selected_ins_sets cannot be empty